This baseline simply passes in our CAPTCHA dataset, and sees how accurately a well-established OCR library can identify it. This will be used as our baseline model for our machine learning solution for CAPTCHA breaking.

In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (3,431 kB/s)
debconf: unable to initi

In [1]:
import pytesseract
import os

# Mount Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
# Sets Kaggle Config Environment Variable (Path of Kaggle.json file)
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Captcha_Datasets"
%cd /content/gdrive/MyDrive/Captcha_Datasets

/content/gdrive/MyDrive/Captcha_Datasets


In [4]:
# Fetch a CAPTCHA Dataset
!kaggle datasets download -d fournierp/captcha-version-2-images

# Then Unzip the Dataset
!unzip \*.zip  && rm *.zip

 92% 16.0M/17.4M [00:00<00:00, 73.7MB/s]
100% 17.4M/17.4M [00:00<00:00, 65.6MB/s]
Archive:  captcha-version-2-images.zip
  inflating: samples/226md.png       
  inflating: samples/22d5n.png       
  inflating: samples/2356g.png       
  inflating: samples/23mdg.png       
  inflating: samples/23n88.png       
  inflating: samples/243mm.png       
  inflating: samples/244e2.png       
  inflating: samples/245y5.png       
  inflating: samples/24f6w.png       
  inflating: samples/24pew.png       
  inflating: samples/25257.png       
  inflating: samples/253dc.png       
  inflating: samples/25egp.png       
  inflating: samples/25m6p.png       
  inflating: samples/25p2m.png       
  inflating: samples/25w53.png       
  inflating: samples/264m5.png       
  inflating: samples/268g2.png       
  inflating: samples/28348.png       
  inflating: samples/28x47.png       
  inflating: samples/2b827.png       
  inflating: samples/2bg48.png       
  inflating: samples/2cegf.png       
  inf

In [3]:
import cv2 as cv
from PIL import Image
import numpy as np

In [ ]:
# Iterate over all files
for files in os.listdir("/content/gdrive/MyDrive/Captcha_Datasets/captcha_images_old"):
  if files.endswith(".png"):    
    res = cv.imread("/content/gdrive/MyDrive/Captcha_Datasets/captcha_images_old/" + files, 0)
    kernel = np.ones((3,3), np.uint8)

    #Operations on the image to cleanup and remove noise
    medBlur = cv.medianBlur(res, 3)
    gaussian_blur = cv.GaussianBlur(medBlur,(5,5),0)
    ret, thresh = cv.threshold(gaussian_blur, 215, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    thresh = cv.dilate(thresh, kernel, iterations=1)
    thresh = cv.erode(thresh, kernel, iterations=1)
    thresh = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=1)

    cv.imwrite("/content/gdrive/MyDrive/Captcha_Datasets/captcha_images_old/processed/" + files, thresh)   
    

In [37]:
# Setup accuracy tracker
total_images = 0;
correct_images = 0;

print("Beginning OCR...")
for files in os.listdir("/content/gdrive/MyDrive/Captcha_Datasets/captcha_images_old/processed"):
  if files.endswith(".png"):
    correct_label = files.split('.')[0]

    image = Image.open("/content/gdrive/MyDrive/Captcha_Datasets/captcha_images_old/processed/" + files)
    # Make pytesseract prediction
    prediction = pytesseract.image_to_string(image)
    if prediction.strip().upper() == correct_label.strip().upper():
      correct_images += 1
    total_images += 1

accuracy = (float(correct_images)/total_images) * 100
print("Total Baseline Model Accuracy: {}%".format(accuracy))

Beginning OCR...
Total Baseline Model Accuracy: 0.4807692307692308%
